### Importación de librerías

In [47]:
#Se importan las librerias a utilizar

import numpy as np
import pandas as pd
import os

import librosa
import librosa.display

----

### Lectura de los datasets

In [48]:
urbansound8k_path = "../../data/raw/UrbanSound8K/"

# Primer dataset - Carpeta con los audios
urbansound8k_audio_path = "audio/"

# Segundo dataset - Archivo CSV con metadatos sobre los audios 
urbansound8k_metadata_path_file = "metadata/UrbanSound8K.csv"

# Dataset CSV con metadatos generados a partir de los audios 
dataset_urbansound8k_path_file = "../../data/processed/cd02_nuevos_metadatos_v2.csv"

In [49]:
dataset_urbansound8k_df = pd.read_csv(urbansound8k_path + urbansound8k_metadata_path_file, sep=",")
dataset_urbansound8k_df.head(2)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing


---
### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos 
4. Formateo definitivo


----

#### Eliminación de columnas 

In [50]:
# Se establece qué columnas se eliminan
columnas_eliminar_lista = ['fsID', 'start', 'end', 'class']

# Se ejecuta la operación
dataset_urbansound8k_df.drop(columnas_eliminar_lista, inplace=True, axis=1) 

In [51]:
print("Vista del dataset de datos:")
display(dataset_urbansound8k_df.head(1))

Vista del dataset de datos:


,slice_file_name,salience,fold,classID
0,100032-3-0-0.wav,1,5,3


In [52]:
print("Columnas del dataset:")
print(dataset_urbansound8k_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID'], dtype='object')
---
Cantidad de columnas: 4
Cantidad de filas: 8732


#### Construcción de atributos MFCC13

In [53]:
def extract_mfcc(file_path, n_mfcc=13):
    signal, sr = librosa.load(file_path, sr=44100, mono=True)  # Cargar el archivo de audio
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)  # Extraer MFCC

    # Calcular estadísticas de los MFCC
    mfcc_mean = np.mean(mfccs, axis=1)  # Promedio de MFCCs
    mfcc_median = np.median(mfccs, axis=1)  # Mediana de MFCCs
    mfcc_std = np.std(mfccs, axis=1)  # Desviación estándar de MFCCs

    # Combinar todas las características en un solo vector
    features = np.hstack([mfcc_mean, mfcc_median, mfcc_std])

    return features

In [54]:

# Crear columnas para cada una de las características
mfcc_columns = [f'mfcc_mean_{i+1}' for i in range(13)] + \
               [f'mfcc_median_{i+1}' for i in range(13)] + \
               [f'mfcc_std_{i+1}' for i in range(13)] 

mfcc_features = []
# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    path_total_file = f"{urbansound8k_path}/audio/fold{row['fold']}/{row['slice_file_name']}"
    mfcc = extract_mfcc(path_total_file)
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_mfcc13_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

In [55]:
dataset_urbansound8k_mfcc13_df.tail(3)

,slice_file_name,salience,fold,classID,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,...,mfcc_std_4,mfcc_std_5,mfcc_std_6,mfcc_std_7,mfcc_std_8,mfcc_std_9,mfcc_std_10,mfcc_std_11,mfcc_std_12,mfcc_std_13
8729,99812-1-4-0.wav,2,7,1,-362.703217,165.081787,-60.780693,30.996469,-9.655406,23.180807,...,17.499111,8.910819,8.859054,10.998036,8.000258,6.817478,4.655785,8.149473,5.556407,5.076985
8730,99812-1-5-0.wav,2,7,1,-404.628113,178.504730,-63.615627,25.944529,-10.428956,12.489921,...,7.955052,6.491368,6.783853,12.653798,10.504695,6.291048,7.251829,8.310383,8.857031,7.287582
8731,99812-1-6-0.wav,2,7,1,-369.637054,147.904556,-63.150425,47.780521,-9.488717,33.391727,...,7.289741,7.708520,6.297567,5.661671,5.337361,5.342536,6.784176,5.789685,4.279145,5.745348


In [56]:
print("Columnas del dataset:")
print(dataset_urbansound8k_mfcc13_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_mfcc13_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_mfcc13_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9',
       'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_mean_13',
       'mfcc_median_1', 'mfcc_median_2', 'mfcc_median_3', 'mfcc_median_4',
       'mfcc_median_5', 'mfcc_median_6', 'mfcc_median_7', 'mfcc_median_8',
       'mfcc_median_9', 'mfcc_median_10', 'mfcc_median_11', 'mfcc_median_12',
       'mfcc_median_13', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3',
       'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8',
       'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12',
       'mfcc_std_13'],
      dtype='object')
---
Cantidad de columnas: 43
Cantidad de filas: 8732


#### Filtrado de filas

In [57]:
filtered_salience_top_1_df = dataset_urbansound8k_mfcc13_df.loc[dataset_urbansound8k_mfcc13_df['salience'] == 1]


print("Campo salience : Una calificación (subjetiva) de la prominencia del sonido. ")
print("Valores: 1 = primer plano, 2 = fondo. ")
print(f"Se filtra los audios con calificación de primer plano. \n")

cantidad_filas_filtradas = dataset_urbansound8k_mfcc13_df.shape[0] - filtered_salience_top_1_df.shape[0]
print(f"Cantidad filas afectadas por filtro: {cantidad_filas_filtradas}")

Campo salience : Una calificación (subjetiva) de la prominencia del sonido. 
Valores: 1 = primer plano, 2 = fondo. 
Se filtra los audios con calificación de primer plano. 

Cantidad filas afectadas por filtro: 3030


### Generación de inputs modelo

#### Dataset input v1 

Todos los audios
Promedio, mediana y desviacion estandar de mfcc13

In [ ]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
columnas_seleccionadas_final = [
       'slice_file_name', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9',
       'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_mean_13',
       'mfcc_median_1', 'mfcc_median_2', 'mfcc_median_3', 'mfcc_median_4',
       'mfcc_median_5', 'mfcc_median_6', 'mfcc_median_7', 'mfcc_median_8',
       'mfcc_median_9', 'mfcc_median_10', 'mfcc_median_11', 'mfcc_median_12',
       'mfcc_median_13', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3',
       'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8',
       'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12',
       'mfcc_std_13'
]

# Crear un nuevo DataFrame con solo las columnas seleccionadas
pd01_transformaciones_v1_df = dataset_urbansound8k_mfcc13_df[columnas_seleccionadas_final]

print(f"Cantidad de columnas del dataset final: {pd01_transformaciones_v1_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {pd01_transformaciones_v1_df.shape[0]}")

pd01_transformaciones_v1_df.to_csv("../../data/processed/pd01_transformaciones_v1.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 42
Cantidad de filas del dataset final: 8732


#### Dataset input v2

audios con salience 1 (calidad más alta)
Promedio, mediana y desviacion estandar de mfcc13

In [ ]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
columnas_seleccionadas_final = [
       'slice_file_name', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9',
       'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_mean_13',
       'mfcc_median_1', 'mfcc_median_2', 'mfcc_median_3', 'mfcc_median_4',
       'mfcc_median_5', 'mfcc_median_6', 'mfcc_median_7', 'mfcc_median_8',
       'mfcc_median_9', 'mfcc_median_10', 'mfcc_median_11', 'mfcc_median_12',
       'mfcc_median_13', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3',
       'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8',
       'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12',
       'mfcc_std_13'
]

# Crear un nuevo DataFrame con solo las columnas seleccionadas
pd01_transformaciones_v2_df = filtered_salience_top_1_df[columnas_seleccionadas_final]

print(f"Cantidad de columnas del dataset final: {pd01_transformaciones_v2_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {pd01_transformaciones_v2_df.shape[0]}")

pd01_transformaciones_v2_df.to_csv("../../data/processed/pd01_transformaciones_v2.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 42
Cantidad de filas del dataset final: 5702
